In [ ]:


!pip install timm
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import pandas as pd
from tqdm import tqdm
import timm
import numpy as np


DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 32
EPOCHS = 60
LR = 1e-3
LR_DECAY = 0.5
PATIENCE = 10
SAVE_PATH_LAT = "best_model_lat_anom.pt"
SAVE_PATH_LONG = "best_model_long_anom.pt"
ANOMALY_IDS = [95, 145, 146, 158, 159, 160, 161]


train_df = pd.read_csv("/kaggle/input/labels-train-only/labels_train (1)(1).csv")
filtered_df = train_df[
    (train_df['latitude'] > 200000) & (train_df['latitude'] < 230000) &
    (train_df['longitude'] > 140000) & (train_df['longitude'] < 150000)
]
filtered_df.to_csv("train_cleaned.csv", index=False)

class LatLongDataset(Dataset):
    def __init__(self, csv_path, tr_path, img_dir, transform=None):
        self.data = pd.read_csv(csv_path)  
        self.datatr = pd.read_csv(tr_path)  
        self.img_dir = img_dir
        self.transform = transform
        
        self.min_lat = min(self.datatr['latitude'])
        self.min_lon = min(self.datatr['longitude'])
        self.max_lat = max(self.datatr['latitude'])
        self.max_lon = max(self.datatr['longitude'])
        
        self.has_labels = 'latitude' in self.data.columns and 'longitude' in self.data.columns

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        img_path = os.path.join(self.img_dir, row['filename'])
        image = Image.open(img_path).convert("RGB")
        
        if self.transform:
            image = self.transform(image)

        if self.has_labels:
            lat = (row['latitude'] - self.min_lat) / (self.max_lat - self.min_lat)
            lon = (row['longitude'] - self.min_lon) / (self.max_lon - self.min_lon)
            return image, torch.tensor(lat, dtype=torch.float), torch.tensor(lon, dtype=torch.float)
        else:
            return image, row['filename']




class EfficientNetLat(nn.Module):
    def __init__(self):
        super(EfficientNetLat, self).__init__()
        self.backbone = timm.create_model("efficientnet_b0", pretrained=True, num_classes=0)
        self.head = nn.Linear(self.backbone.num_features, 128)
        self.head1 = nn.Linear(128, 1)
        self.sigmoid = nn.Sigmoid()  

    def forward(self, x):
        x = self.backbone(x)
        x = self.head(x)
        x = self.head1(x)
        x = self.sigmoid(x)  
        return x

class EfficientNetLong(nn.Module):
    def __init__(self):
        super(EfficientNetLong, self).__init__()
        self.backbone = timm.create_model("efficientnet_b0", pretrained=True, num_classes=0)
        self.head = nn.Linear(self.backbone.num_features, 128)
        self.head1 = nn.Linear(128, 1)
        self.sigmoid = nn.Sigmoid()  

    def forward(self, x):
        x = self.backbone(x)
        x = self.head(x)
        x = self.head1(x)
        x = self.sigmoid(x)  
        return x



transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.RandomRotation(30),
    transforms.RandomHorizontalFlip(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


train_dataset = LatLongDataset(
    "/kaggle/working/train_cleaned.csv",  
    "/kaggle/working/train_cleaned.csv",  
    "/kaggle/input/mydataset/images_train/images_train/images_train",  
    transform
)


train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)

val_df = pd.read_csv("/kaggle/input/mydataset/labels_val.csv")
val_df_cleaned = val_df.drop(index=ANOMALY_IDS)
val_df_cleaned.to_csv("val_cleaned.csv", index=False)

val_dataset = LatLongDataset(
    "/kaggle/working/val_cleaned.csv",
    "/kaggle/working/val_cleaned.csv",
    "/kaggle/input/mydataset/images_val/images_val",
    transform
)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)


modellat = EfficientNetLat().to(DEVICE)
modellong = EfficientNetLong().to(DEVICE)
optimizerlat = optim.Adam(modellat.parameters(), lr=LR)
optimizerlong = optim.Adam(modellong.parameters(), lr=LR)
schedulerlat = optim.lr_scheduler.StepLR(optimizerlat, step_size=5, gamma=LR_DECAY)
schedulerlong = optim.lr_scheduler.StepLR(optimizerlong, step_size=5, gamma=LR_DECAY)
criterion = nn.MSELoss()

best_val_loss = float('inf')
patience_counter = 0

for epoch in range(EPOCHS):
    modellat.train()
    modellong.train()
    train_losses = []

    for images, lat, long in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
        images, lat, long = images.to(DEVICE), lat.to(DEVICE), long.to(DEVICE)

        optimizerlat.zero_grad()
        optimizerlong.zero_grad()

        pred_lat = modellat(images)
        pred_long = modellong(images)

        loss_lat = criterion(pred_lat, lat.view(-1, 1))
        loss_long = criterion(pred_long, long.view(-1, 1))

        loss_lat.backward()
        loss_long.backward()

        optimizerlat.step()
        optimizerlong.step()

        train_losses.append((loss_lat.item() + loss_long.item()) / 2)

    schedulerlat.step()
    schedulerlong.step()
    avg_train_loss = np.mean(train_losses)



    modellat.eval()
    modellong.eval()
    val_losses = []

    with torch.no_grad():
        for images, lat, long in val_loader:
            images, lat, long = images.to(DEVICE), lat.to(DEVICE), long.to(DEVICE)
            pred_lat = modellat(images)
            pred_long = modellong(images)
            loss_lat = criterion(pred_lat, lat.view(-1, 1))
            loss_long = criterion(pred_long, long.view(-1, 1))
            val_losses.append((loss_lat.item() + loss_long.item()) / 2)

    avg_val_loss = np.mean(val_losses)
    print(f"Epoch {epoch+1}: Train Loss = {avg_train_loss:.4f}, Val Loss = {avg_val_loss:.4f}")

    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        patience_counter = 0
        torch.save(modellat.state_dict(), SAVE_PATH_LAT)
        torch.save(modellong.state_dict(), SAVE_PATH_LONG)
    else:
        patience_counter += 1
        if patience_counter >= PATIENCE:
            print("Early stopping triggered.")
            break


modellat.load_state_dict(torch.load(SAVE_PATH_LAT))
modellong.load_state_dict(torch.load(SAVE_PATH_LONG))
modellat.eval()
modellong.eval()



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 29.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 72.9 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.8.93
    Uninstalling nvidia-nvjitlink-cu12-12.8.93:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.8.93
  Attempting uninstall: nvidia-curand-cu12
    Found existing installation: nvidia-curand-cu12 10.3.9.90
    Uninstalling nvidia-curand-cu12-1

model.safetensors:   0%|          | 0.00/21.4M [00:00<?, ?B/s]

Epoch 1: 100%|██████████| 203/203 [00:50<00:00,  4.03it/s]


Epoch 1: Train Loss = 0.0282, Val Loss = 0.0236


Epoch 2: 100%|██████████| 203/203 [00:46<00:00,  4.33it/s]


...

Epoch 43: Train Loss = 0.0004, Val Loss = 0.0032


Epoch 44: 100%|██████████| 203/203 [00:46<00:00,  4.35it/s]


Epoch 44: Train Loss = 0.0004, Val Loss = 0.0033
Early stopping triggered.


/tmp/ipykernel_31/2619870465.py:223: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  modellat.load_state_dict(torch.load(SAVE_PATH_LAT))
/tmp/ipykernel_31/2619870465.py:224: F

EfficientNetLong(
  (backbone): EfficientNet(
    (conv_stem): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn1): BatchNormAct2d(
      32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
      (drop): Identity()
      (act): SiLU(inplace=True)
    )
    (blocks): Sequential(
      (0): Sequential(
        (0): DepthwiseSeparableConv(
          (conv_dw): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (bn1): BatchNormAct2d(
            32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
            (drop): Identity()
            (act): SiLU(inplace=True)
          )
          (aa): Identity()
          (se): SqueezeExcite(
            (conv_reduce): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (act1): SiLU(inplace=True)
            (conv_expand): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (gate): Sigmoid()
          )
          (conv_p

In [ ]:

modellat.load_state_dict(torch.load(SAVE_PATH_LAT))
modellong.load_state_dict(torch.load(SAVE_PATH_LONG))
modellat.eval()
modellong.eval()

predlats, predlongs = [], []
with torch.no_grad():
    for images, _, _ in val_loader:
        images = images.to(DEVICE)
        pred_lat = modellat(images).cpu().numpy()
        pred_lon = modellong(images).cpu().numpy()

        pred_lat = pred_lat * (train_dataset.max_lat - train_dataset.min_lat) + train_dataset.min_lat
        pred_lon = pred_lon * (train_dataset.max_lon - train_dataset.min_lon) + train_dataset.min_lon

        predlats.extend(pred_lat.squeeze().tolist())
        predlongs.extend(pred_lon.squeeze().tolist())

submission_val = pd.DataFrame({
    "id": val_df_cleaned.index,
    "Latitude": predlats,
    "Longitude": predlongs
})



import os

test_img_dir = "/kaggle/input/mydataset2/images_test"
test_filenames = os.listdir(test_img_dir)

test_df = pd.DataFrame({"filename": test_filenames})
test_df.to_csv("test_dummy.csv", index=False)

test_dataset = LatLongDataset(
    "test_dummy.csv",                  
    "/kaggle/working/train_cleaned.csv",  
    test_img_dir,
    transform
)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

test_preds_lat, test_preds_lon = [], []
with torch.no_grad():
    for images, _ in tqdm(test_loader, desc="Predicting test set"):
        images = images.to(DEVICE)
        pred_lat = modellat(images).cpu().numpy()
        pred_lon = modellong(images).cpu().numpy()

        pred_lat = pred_lat * (train_dataset.max_lat - train_dataset.min_lat) + train_dataset.min_lat
        pred_lon = pred_lon * (train_dataset.max_lon - train_dataset.min_lon) + train_dataset.min_lon

        test_preds_lat.extend(pred_lat.squeeze().tolist())
        test_preds_lon.extend(pred_lon.squeeze().tolist())

submission_test = pd.DataFrame({
    "id": range(len(val_df), len(val_df) + len(test_preds_lat)),
    "Latitude": test_preds_lat,
    "Longitude": test_preds_lon
})


final_submission = pd.concat([submission_val, submission_test], ignore_index=True)
final_submission.to_csv("/kaggle/working/final_submission.csv", index=False)
print("✅ Final submission saved with full predictions (val + test).")

/tmp/ipykernel_31/2399401389.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  modellat.load_state_dict(torch.load(SAVE_PATH_LAT))
/tmp/ipykernel_31/2399401389.py:5: Futur

✅ Final submission saved with full predictions (val + test).
